In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# hyper parameters

num_epochs = 80

learning_rate = 0.001

In [5]:
# image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [6]:
# MNIST dataset(image and labels)

train_dataset = torchvision.datasets.CIFAR10(root = '../../data',
                                         train = True,
                                         transform=transforms.ToTensor(),
                                         download=True
                                         )

test_dataset = torchvision.datasets.CIFAR10(root = '../../data',
                                         train = False,
                                         transform=transforms.ToTensor(),
                                         )

# data loader(input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

Files already downloaded and verified


NameError: name 'batch_size' is not defined

In [13]:
# Define your model below:

# 3x3 convulotion
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# residual net
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.fc = nn.Linear(64, num_classes)
        
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsaple = nn.Sequential(
            conv3x3(self.in_channels, out_channels, stride=stride),
            nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return x

    
model = ResidualBlock(ResidualBlock, [2, 2, 2]).to(device)

TypeError: unsupported operand type(s) for %: 'type' and 'int'

In [8]:
# loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2292
Epoch [1/5], Step [200/600], Loss: 0.0839
Epoch [1/5], Step [300/600], Loss: 0.1572
Epoch [1/5], Step [400/600], Loss: 0.0953
Epoch [1/5], Step [500/600], Loss: 0.0319
Epoch [1/5], Step [600/600], Loss: 0.0222
Epoch [2/5], Step [100/600], Loss: 0.0503
Epoch [2/5], Step [200/600], Loss: 0.0453
Epoch [2/5], Step [300/600], Loss: 0.0548
Epoch [2/5], Step [400/600], Loss: 0.0484
Epoch [2/5], Step [500/600], Loss: 0.0267
Epoch [2/5], Step [600/600], Loss: 0.0183
Epoch [3/5], Step [100/600], Loss: 0.0367


In [ ]:
# test the model
# in test phase, don't compute gradients for memory efficiency

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # images = images.reshape(-1, 28*28)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    



In [12]:
torch.save(model.state_dict(), 'Deep Residual Network.ckpt')

NameError: name 'model' is not defined

In [21]:
Dict = {
    'a': 1,
    'b': 2,
    'c': 3
}

In [27]:
max(Dict, key=Dict.get)

'c'

In [20]:
Dict[max(Dict)]

3

In [18]:
Dict.pop(max(Dict))
Dict

{'a': 1, 'b': 2}